## Part 1: ML Model

In [ ]:
!pip3 install tensorflow
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split
from glob import glob

# --- Parameters ---
IMG_SIZE = (128, 128)  # Resize all images to this size
DATA_PATH = "./data"   # Path to the dataset folder
EPOCHS = 20
BATCH_SIZE = 32


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


2024-12-08 14:14:57.522710: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 14:14:57.770663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733685297.877334   89439 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733685297.909853   89439 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 14:14:58.083056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# --- Function to Load and Preprocess Images ---
def load_images(data_path, img_size):
    images = []
    labels = []
    
    for label, category in enumerate(["0min", "5min", "15min", "30min"]):
        category_path = os.path.join(data_path, category)
        print(F"Reading category {category}")
        for dapi_img_path in glob(os.path.join(category_path, "*_[0-9][0-9][0-9][0-9]_DAPI.tif")):
            trans_img_path = dapi_img_path.replace("_DAPI.tif", "_TRANS.tif")
            if not os.path.exists(trans_img_path):
                continue
            
            # Load DAPI and Trans images
            dapi_img = cv2.imread(dapi_img_path, cv2.IMREAD_GRAYSCALE)
            trans_img = cv2.imread(trans_img_path, cv2.IMREAD_GRAYSCALE)
            
            # Resize images to the same size
            dapi_img = cv2.resize(dapi_img, img_size)
            trans_img = cv2.resize(trans_img, img_size)
            
            # Stack as two-channel image
            combined_img = np.stack((dapi_img, trans_img), axis=0)
            
            images.append(combined_img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# --- Load Data ---
print("Loading images...")
X, y = load_images(DATA_PATH, IMG_SIZE)

X, y, len(X), X.shape

Loading images...
Reading category 0min
Reading category 5min
Reading category 15min
Reading category 30min


(array([[[[  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          ...,
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2]],
 
         [[184, 184, 183, ..., 192, 191, 185],
          [181, 184, 184, ..., 185, 184, 177],
          [182, 186, 186, ..., 179, 182, 173],
          ...,
          [184, 186, 185, ..., 191, 190, 191],
          [186, 187, 185, ..., 190, 193, 190],
          [183, 187, 185, ..., 191, 190, 189]]],
 
 
        [[[  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          ...,
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2],
          [  2,   2,   2, ...,   2,   2,   2]],
 
         [[183, 185, 182, ..., 180, 182, 180],
          [181, 184, 182, ..., 182, 181, 177],
   

In [ ]:
# Normalize images to [0, 1] range
X = X / 255.0
y = to_categorical(y)  # One-hot encoding

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Validation samples: {X_val.shape[0]}, Test samples: {X_test.shape[0]}")

# --- Build CNN Model ---
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(2, IMG_SIZE[0], IMG_SIZE[1])),  # 2 channels for DAPI+Trans
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')  # Output layer (number of classes)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --- Train Model ---
print("Training model...")
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE)

# --- Evaluate Model ---
print("Evaluating model...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# --- Save Model ---
model.save("uv_classification_model.h5")
print("Model saved as uv_classification_model.h5")

Training samples: 45, Validation samples: 10, Test samples: 10


ValueError: Computed output size would be negative. Received `inputs shape=(None, 0, 63, 32)`, `kernel shape=(3, 3, 32, 64)`, `dilation_rate=[1 1]`.